# Extraction de données : méthode  du scrapping

# Quelques règles à respecter 

Pour faire du web scraping, il y a des règles à respecter ! Le web scraping peut poser des problèmes si vous récupérez des informations sans respecter les conditions d’utilisation du site.
Voici les bonnes pratiques légales :        
● Consulter le fichier robots.txt d’un site (situé à https://<nomdusite.com>/robots.txt). Ce fichier indique quelles parties du site sont autorisées ou interdites au scraping.         
● Ne pas surcharger un site en envoyant trop de requêtes à la fois (cela peut le ralentir ou le bloquer).           
● Ne pas collecter de données personnelles sans consentement (ex. : adresses e-mail, noms de personnes).            
● Vérifier les conditions d'utilisation du site pour voir si le scraping est autorisé.          


# Comprendre le code HTML

Pour bien comprendre le code html, j'ai créé ma première page web : ma_page.html. ce dodument peut être ouvert avc n'mporte quel navigateur web


In [47]:
# 1. <!DOCTYPE html> : Indique que le document est une page web HTML.
# 2. <html> : Balise racine qui englobe tout le contenu.
# 3. <head> : Contient les informations sur la page (titre, styles, etc.).
# 4. <title> : Définit le titre de l’onglet du navigateur.
# 5. <body> : Contient le contenu visible de la page.
# 6. <h1> : Un gros titre.
# 7. <p> : Un paragraphe de texte.
# 8. <a href=""> : Un lien vers un autre site.
# 9. <ul> et <li> : Une liste non ordonnée avec des éléments.

● Une classe (class) permet d’appliquer le même style à plusieurs éléments.     
● Un identifiant (id) est unique et permet d’identifier un seul élément spécifique.     ![alt text](image-1.png)

# Exploration d'une page web

1. Allez sur https://books.toscrape.com/.           
2. Faites cliquer droit > Inspecter l’élément (ou appuyez sur F12).     
3. Survolez les titres des livres et observez leur structure HTML.          

![alt text](image-2.png)

On va inspecter plus particulièrement le premier livre : 
![alt text](image-3.png)



4. Identifiez les balises qui contiennent les noms, les images, ainsi que les prix des livres.
    - Le nom du livre est situé au-dessus tout au dessus dans "head" et title" où nous avons donc tout le résumé du bouquin.

    - Pour toruver l'image correspondante il faut chercher plus bas dans les balises : l'image est situé 
    ![alt text](image-4.png)

    - Pour trouver le prix du livre, c'est à la suite des balises des images !
    [alt text](image-5.png)

NB : le prix, le nombre de livre en stock, le nombre d'étoile et le message warning sont tous répertoriés dans la class "col-sm-6 product main"
![alt text](image-6.png)

Il est aussi possible d'avoir toutes ces données directement pour chaque livre comme par exemple. toutes ces informations se trouves dans la classe 'product_pod"
- image : class : image-container 
- note : class : class_rating Three
- prix : class : product_price
![alt text](image-7.png)


5. Remarquez que tous les livres appartiennent à une même class. Identifiez-la.     

 - body id = "default", class : "default"
  puis il y a un autre niveau "article", class_="product_pod" qui contient comlme on a vu précédemement toutes informations des notes et des prix



# Beautiful et requests

avec la requête "requests" cela nous permet d'extraire le contenu d'une âge html mais le résultat n'est pas du tout optimal

In [48]:
import requests

url = "https://books.toscrape.com/"
response = requests.get(url)

In [49]:
print(response.status_code) # Affiche si la requête a réussi (200)
response.text[:500]  # Affiche les 500 premiers caractères du HTML

200


'<!DOCTYPE html>\n<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->\n<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->\n<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->\n<!--[if gt IE 8]><!--> <html lang="en-us" class="no-js"> <!--<![endif]-->\n    <head>\n        <title>\n    All products | Books to Scrape - Sandbox\n</title>\n\n        <meta http-equiv="content-type" content="text/html; charset=UTF-8" /'

Beautifusoup va nous permettre de d'analyser et d'extraire des éléments souhaités en utilisant les baslises html.

In [50]:
import requests
from bs4 import BeautifulSoup

url = "https://books.toscrape.com/"
page = requests.get(url)

soup = BeautifulSoup(page.content, "html.parser")

Utilisez requests et Beautifulsoup pour extraire uniquement les livres les mieux notés (>4 étoiles) et moins chers que 30£ présentés sur les 3 premières pages. Enregistrez le résultat dans un fichier csv

In [51]:
import re
from urllib.parse import urljoin

import requests
from bs4 import BeautifulSoup

url = "https://books.toscrape.com/"

resultats=[] # Liste pour stocker les données extraites

for page_num in range(1, 4):  # Scraper les 3 premières pages
    #page 1 = index.html ; page 2 = catalogue/page-2.html
    if page == 1:
        page_url = urljoin(url, "index.html")
    else:
        page_url = urljoin(url, f"catalogue/page-{page_num}.html")

    req = requests.get(page_url) 
    req.raise_for_status()  # Vérifie que la requête a réussi           
    soup = BeautifulSoup(req.content, "html.parser")

## deux choix de sélection des livres 
    livres = soup.find_all("article", class_="product_pod") ## permet de récupérer tous les livres
    # ou livres = soup.select("article.product_pod") 
    
    for livre in livres:
        
        # extraire les titres
        titre = livre.h3.a["title"]
        
        # extraire les prix
        prix_txt= livre.find("p", class_="price_color").text
        m = re.search(r"£([0-9]+\.[0-9]+)", prix_txt) ## permet d'extraire le prix numérique
        prix= float(m.group(1)) if m else None # permet de convertir en float
        
        # extraire les notes des livres (1 à 5)
        notes_txt = livre.find("p", class_="star-rating")["class"]
        classes= ["One", "Two", "Three", "Four", "Five"]
        note = None       
        if notes_txt:
            for cls in classes:
                if cls in notes_txt:
                    note = classes.index(cls) + 1
                    break
        
        # extraire l'url de la page du livre
        href = livre.h3.a["href"]
        book_url = urljoin(page_url, href)
        
        # Stocker les données extraites dans un dictionnaire
        if note >4 and (prix is not None) and (prix < 30):
            resultat = {
                "titre": titre,
                "prix": prix,
                "note": note,
                "url": book_url
            }
            resultats.append(resultat)

In [52]:
resultats

[{'titre': 'Set Me Free',
  'prix': 17.46,
  'note': 5,
  'url': 'https://books.toscrape.com/catalogue/set-me-free_988/index.html'},
 {'titre': 'Chase Me (Paris Nights #2)',
  'prix': 25.27,
  'note': 5,
  'url': 'https://books.toscrape.com/catalogue/chase-me-paris-nights-2_977/index.html'},
 {'titre': 'The Four Agreements: A Practical Guide to Personal Freedom',
  'prix': 17.66,
  'note': 5,
  'url': 'https://books.toscrape.com/catalogue/the-four-agreements-a-practical-guide-to-personal-freedom_970/index.html'},
 {'titre': 'The Elephant Tree',
  'prix': 23.82,
  'note': 5,
  'url': 'https://books.toscrape.com/catalogue/the-elephant-tree_968/index.html'},
 {'titre': "Sophie's World",
  'prix': 15.94,
  'note': 5,
  'url': 'https://books.toscrape.com/catalogue/sophies-world_966/index.html'},
 {'titre': '#HigherSelfie: Wake Up Your Life. Free Your Soul. Find Your Tribe.',
  'prix': 23.11,
  'note': 5,
  'url': 'https://books.toscrape.com/catalogue/higherselfie-wake-up-your-life-free-your

In [53]:
import csv
with open("results_books_save.csv", "w", newline="", encoding="utf-8") as csvfile:
    fieldnames = ["titre", "prix", "note", "url"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    writer.writerows(resultats)
    
print(f"[OK] {len(resultats)} livres exportés -> results_books_save.csv")

[OK] 7 livres exportés -> results_books_save.csv


Question BONUS  :     
Créez un classement des 10 catégories ayant les livres les plus chers en moyenne.


In [54]:
## Pour récupérer les catégories des livres, on va parcourir le menu des catégories sur la page principale.

# Menu catégories: ul.nav-list > li > ul > li > a
cat_links = soup.select("ul.nav-list ul li a")

categories = []
for a in cat_links:
    name = a.get_text(strip=True)
    href = a.get("href")
    cat_url = urljoin(url, "catalogue/"+ href)
    categories.append((name, cat_url))
    
print(f"{len(categories)} catégories trouvées")
categories[:10]


50 catégories trouvées


[('Travel',
  'https://books.toscrape.com/catalogue/category/books/travel_2/index.html'),
 ('Mystery',
  'https://books.toscrape.com/catalogue/category/books/mystery_3/index.html'),
 ('Historical Fiction',
  'https://books.toscrape.com/catalogue/category/books/historical-fiction_4/index.html'),
 ('Sequential Art',
  'https://books.toscrape.com/catalogue/category/books/sequential-art_5/index.html'),
 ('Classics',
  'https://books.toscrape.com/catalogue/category/books/classics_6/index.html'),
 ('Philosophy',
  'https://books.toscrape.com/catalogue/category/books/philosophy_7/index.html'),
 ('Romance',
  'https://books.toscrape.com/catalogue/category/books/romance_8/index.html'),
 ('Womens Fiction',
  'https://books.toscrape.com/catalogue/category/books/womens-fiction_9/index.html'),
 ('Fiction',
  'https://books.toscrape.com/catalogue/category/books/fiction_10/index.html'),
 ('Childrens',
  'https://books.toscrape.com/catalogue/category/books/childrens_11/index.html')]

In [55]:
# Pour chaque catégorie, stock les prix
sum_by_cat = {}
count_by_cat = {}

for (cat_name, cat_url) in categories:
    sum_by_cat[cat_name] = 0.0
    count_by_cat[cat_name] = 0




    next_url = cat_url

    while next_url:
        rc = requests.get(next_url)
        sc = BeautifulSoup(rc.text, "html.parser")

        books = sc.select("article.product_pod")
        for b in books:
            price_txt = b.select_one("p.price_color").get_text(strip=True)  ## une autre facçon d'extraire le prix différente que précédemment
            m = re.search(r"(\d+(?:\.\d+)?)", price_txt)
            if not m:
                continue
            price = float(m.group(1))
            sum_by_cat[cat_name] += price
            count_by_cat[cat_name] += 1

        # pagination dans la catégorie: li.next a => permet de passer à la page suivante
        next_a = sc.select_one("li.next a")
        if next_a:
            next_href = next_a.get("href")
            next_url = urljoin(next_url, next_href)  # important: base = page courante
        else:
            next_url = None


In [56]:
print(sum_by_cat)
print(count_by_cat)

{'Travel': 437.74, 'Mystery': 1015.01, 'Historical Fiction': 874.7500000000001, 'Sequential Art': 2592.9200000000005, 'Classics': 694.36, 'Philosophy': 369.14, 'Romance': 1187.6799999999998, 'Womens Fiction': 625.45, 'Fiction': 2344.3300000000004, 'Childrens': 946.51, 'Religion': 227.97000000000003, 'Nonfiction': 3768.6200000000003, 'Music': 463.28000000000003, 'Default': 5227.69, 'Science Fiction': 540.84, 'Sports and Games': 205.82999999999998, 'Add a comment': 2398.3599999999997, 'Fantasy': 1900.51, 'New Adult': 278.3, 'Young Adult': 1914.25, 'Science': 463.24, 'Poetry': 683.51, 'Paranormal': 15.4, 'Art': 308.15999999999997, 'Psychology': 239.53, 'Autobiography': 333.48, 'Parenting': 37.35, 'Adult Fiction': 15.36, 'Humor': 335.01, 'Horror': 611.14, 'History': 671.31, 'Food and Drink': 942.4399999999999, 'Christian Fiction': 206.31, 'Business': 389.52000000000004, 'Biography': 168.31, 'Thriller': 345.77, 'Contemporary': 108.6, 'Spirituality': 210.59, 'Academic': 13.12, 'Self Help': 2

In [57]:
## afficher la moyenne des prix par catégorie

avg_list = []
for cat_name in sum_by_cat:
    n = count_by_cat[cat_name]
    avg = sum_by_cat[cat_name] / n
    avg_list.append((cat_name, avg, n))

avg_list.sort(key=lambda x: x[1], reverse=True) ## permet de trier par moyenne décroissante
top10 = avg_list[:10]

top10

[('Suspense', 58.33, 1),
 ('Novels', 54.81, 1),
 ('Politics', 53.61333333333332, 3),
 ('Health', 51.4525, 4),
 ('New Adult', 46.38333333333333, 6),
 ('Christian', 42.49666666666666, 3),
 ('Sports and Games', 41.166, 5),
 ('Self Help', 40.620000000000005, 5),
 ('Travel', 39.79454545454546, 11),
 ('Fantasy', 39.59395833333333, 48)]

In [58]:
# bel afficahge
print("Top 10 des catégories avec la moyenne des prix les plus élevées:")
for i, (cat, avg, n) in enumerate(top10, start=1):
    print(f"{i:>2}. {cat:<25} moyenne = £{avg:.2f} (n={n})")

Top 10 des catégories avec la moyenne des prix les plus élevées:
 1. Suspense                  moyenne = £58.33 (n=1)
 2. Novels                    moyenne = £54.81 (n=1)
 3. Politics                  moyenne = £53.61 (n=3)
 4. Health                    moyenne = £51.45 (n=4)
 5. New Adult                 moyenne = £46.38 (n=6)
 6. Christian                 moyenne = £42.50 (n=3)
 7. Sports and Games          moyenne = £41.17 (n=5)
 8. Self Help                 moyenne = £40.62 (n=5)
 9. Travel                    moyenne = £39.79 (n=11)
10. Fantasy                   moyenne = £39.59 (n=48)
